In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master('local[*]')\
        .appName('test') \
        .getOrCreate()

In [ ]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [ ]:
common_cols = []
yellow_cols = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_cols:
        common_cols.append(col)

In [ ]:
from pyspark.sql import functions as funcs

In [ ]:
df_green_sel = df_green.select(common_cols)
df_yellow_sel = df_yellow.select(common_cols)

In [ ]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [ ]:
df_results = spark.sql("""
SELECT  
    -- Revenue grouping 
    date_trunc('month', 'pickup_datetime') as revenue_month, 

    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) as revenue_monthly_fare,
    SUM(extra) as revenue_monthly_extra,
    SUM(mta_tax) as revenue_monthly_mta_tax,
    SUM(tip_amount) as revenue_monthly_tip_amount,
    SUM(tolls_amount) as revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) as revenue_monthly_improvement_surcharge,
    SUM(total_amount) as revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) as avg_monthly_passenger_count,
    AVG(trip_distance) as avg_monthly_trip_distance

FROM trips_data
    GROUP BY revenue_month, service_type

        """)

In [ ]:
df_results.show()